In [18]:
# kaggle Ddos dataseti ile ctgan çalışması

# Colab: CTGAN ile hızlı prototip
!pip install -q ctgan pandas scikit-learn matplotlib seaborn umap-learn

import pandas as pd, numpy as np

In [22]:
pip install ctgan

In [2]:
# DATASET DOSYASINI GOOGLE DRİVE /tmp klasörüne unzip edilmesi
import shutil
from google.colab import drive

drive.mount("/content/gdrive")
# Change the code below if the path to the dataset is different for you.
shutil.unpack_archive("/content/gdrive/MyDrive/ddos_dataset.zip", "/tmp/")

Mounted at /content/gdrive


In [3]:
# 1) CSV yükle (yolunu değiştir)
csv_path = r"/tmp/dataset_sdn.csv"   # kendi dosya yolunu yaz
df = pd.read_csv(csv_path, low_memory=False)
print(df.shape)
display(df.head())

(104345, 23)


,dt,switch,src,dst,pktcount,bytecount,dur,dur_nsec,tot_dur,flows,...,pktrate,Pairflow,Protocol,port_no,tx_bytes,rx_bytes,tx_kbps,rx_kbps,tot_kbps,label
0,11425,1,10.0.0.1,10.0.0.8,45304,48294064,100,716000000,1.010000e+11,3,...,451,0,UDP,3,143928631,3917,0,0.0,0.0,0
1,11605,1,10.0.0.1,10.0.0.8,126395,134737070,280,734000000,2.810000e+11,2,...,451,0,UDP,4,3842,3520,0,0.0,0.0,0
2,11425,1,10.0.0.2,10.0.0.8,90333,96294978,200,744000000,2.010000e+11,3,...,451,0,UDP,1,3795,1242,0,0.0,0.0,0
3,11425,1,10.0.0.2,10.0.0.8,90333,96294978,200,744000000,2.010000e+11,3,...,451,0,UDP,2,3688,1492,0,0.0,0.0,0
4,11425,1,10.0.0.2,10.0.0.8,90333,96294978,200,744000000,2.010000e+11,3,...,451,0,UDP,3,3413,3665,0,0.0,0.0,0


In [4]:
# 2) Hedef ve ön-işlem (label tespiti, örnek)
target = None
for t in ['label','Label','class','Class','attack','Attack','target','Target','flow_label']:
    if t in df.columns:
        target = t
        break
if target is None:
    target = df.columns[-1]
print("target:", target)

# Basit: hedef numeric değilse encode et
y_raw = df[target]
if y_raw.dtype == object:
    df[target] = pd.factorize(y_raw)[0]

target: label


In [5]:
# 3) Kategorik değişkenleri tespit et (ctgan için kategori listesi oluştur)
cat_cols = df.select_dtypes(include=['object','category']).columns.tolist()
# Eğer label kategorikse cat_cols'a dahil etme (ctgan'da label'i conditional kullanacağız)
if target in cat_cols:
    cat_cols.remove(target)
print("categorical columns:", cat_cols)

categorical columns: ['src', 'dst', 'Protocol']


In [6]:
# 4) Feature selection (opsiyonel) - örnek: RandomForest importance ile hızlı seçim
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
X = df.drop(columns=[target])
y = df[target]
# Basit numeric-only for RF feature importance (önce encoding)
X_enc = pd.get_dummies(X, drop_first=True)
X_train, X_val, y_train, y_val = train_test_split(X_enc, y, test_size=0.2, random_state=42, stratify=y)
rf = RandomForestClassifier(n_estimators=200, n_jobs=-1, random_state=42, class_weight='balanced')
rf.fit(X_train, y_train)
imp = pd.Series(rf.feature_importances_, index=X_enc.columns).sort_values(ascending=False)
top_features = imp.head(10).index.tolist()   # örnek: top 10 seç
print("Top features seçildi:", len(top_features))


Top features seçildi: 10


In [11]:

#Top features
print(top_features)

['byteperflow', 'pktperflow', 'pktrate', 'bytecount', 'pktcount', 'packetins', 'tot_dur', 'dur', 'dt', 'Protocol_TCP']


In [27]:
# 4) Sadece seçilen kolonlar + target'ı al
selected_features = [
    'dt','pktcount','bytecount','dur','tot_dur',
    'packetins','pktperflow','byteperflow','pktrate','tx_bytes'
]
use_cols = [c for c in selected_features if c in df.columns] + [target]
df_sel = df[use_cols].dropna()
print("Kullanılan kolonlar:", df_sel.columns.tolist())
print(df_sel.head())

Kullanılan kolonlar: ['dt', 'pktcount', 'bytecount', 'dur', 'tot_dur', 'packetins', 'pktperflow', 'byteperflow', 'pktrate', 'tx_bytes', 'label']
      dt  pktcount  bytecount  dur       tot_dur  packetins  pktperflow  \
0  11425     45304   48294064  100  1.010000e+11       1943       13535   
1  11605    126395  134737070  280  2.810000e+11       1943       13531   
2  11425     90333   96294978  200  2.010000e+11       1943       13534   
3  11425     90333   96294978  200  2.010000e+11       1943       13534   
4  11425     90333   96294978  200  2.010000e+11       1943       13534   

   byteperflow  pktrate   tx_bytes  label  
0     14428310      451  143928631      0  
1     14424046      451       3842      0  
2     14427244      451       3795      0  
3     14427244      451       3688      0  
4     14427244      451       3413      0  


In [29]:
# 5) Kategorik kolonları bul (ctgan için lazım)
cat_cols = df_sel.select_dtypes(include=['object','category']).columns.tolist()
if target in cat_cols:
    cat_cols.remove(target)   # label'i conditional yapacağız
print("Categorical columns:", cat_cols)

Categorical columns: []


In [30]:
# 6) Eğer target kategorik ise factorize et (numeric label olsun)
if df_sel[target].dtype == object:
    df_sel[target] = pd.factorize(df_sel[target])[0]

In [33]:
# 7) CTGAN eğitimi
from ctgan import CTGAN

ctgan = CTGAN(epochs=5)  # 300 epoch örnek, hızlı denemelerde 50-100 yapabilirsin
ctgan.fit(df_sel, discrete_columns=cat_cols)



In [34]:
# 8) Sentetik veri üret
synthetic = ctgan.sample(len(df_sel))
print("Synthetic shape:", synthetic.shape)
print(synthetic.head())

Synthetic shape: (104345, 11)
      dt  pktcount  bytecount   dur       tot_dur  packetins  pktperflow  \
0  32657       626    -346417  1135 -2.476306e+10       2755          80   
1   7566      -867       6486  1074 -4.599749e+10       4558           1   
2  32636      1533     304685   751  1.858852e+10       2522         155   
3   6003    186284   43381955    92  1.923851e+11       -634       18928   
4  11850     85167   18023415   241  3.167993e+11      25009        6657   

   byteperflow  pktrate   tx_bytes  label  
0        25315        0  137656314      0  
1       101069       -3  497869640      0  
2       147304       -3  138518596      0  
3       -37208      119  525329488      0  
4       325791      228   12009132      1  
